In [1]:
# Import required libraries and dependencies
import pandas as pd
import numpy as np
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import holoviews as hv
from holoviews import dim, opts

# import bokeh extension for holoviews
hv.extension('bokeh')

# Supress Warnings
import warnings
warnings.simplefilter(action = "ignore")

In [4]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Define a dictionary structure for header row styling
df_header_row = {
    'selector': 'th.col_heading',
    'props': 'background-color: bisque;'
}

# Display the DataFrame with the index value
df_market_data.head(5).style.set_table_styles([df_header_row])

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.083880,7.602780,6.575090,7.672580,-3.251850,83.518400,37.517610
ethereum,0.223920,10.381340,4.808490,0.131690,-12.888900,186.774180,101.960230
tether,-0.211730,0.049350,0.006400,-0.042370,0.280370,-0.005420,0.019540
ripple,-0.378190,-0.609260,2.249840,0.234550,-17.552450,39.538880,-16.601930
bitcoin-cash,2.905850,17.097170,14.753340,15.749030,-13.717930,21.660420,14.493840


In [5]:
# Generate summary statistics and display the dataframe with the index value
df_market_data.head(5).style.set_table_styles([df_header_row])

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.083880,7.602780,6.575090,7.672580,-3.251850,83.518400,37.517610
ethereum,0.223920,10.381340,4.808490,0.131690,-12.888900,186.774180,101.960230
tether,-0.211730,0.049350,0.006400,-0.042370,0.280370,-0.005420,0.019540
ripple,-0.378190,-0.609260,2.249840,0.234550,-17.552450,39.538880,-16.601930
bitcoin-cash,2.905850,17.097170,14.753340,15.749030,-13.717930,21.660420,14.493840


In [6]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

### Prepare the Data

In [7]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
crypto_scaled = StandardScaler().fit_transform(df_market_data)
crypto_scaled[0:3]

array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ,
        -0.35595348, -0.25163688],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273,
        -0.11575947, -0.19935211],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452,
        -0.55024692, -0.28206051]])

In [21]:
# Create a DataFrame with the scaled data
crypto_transformed = pd.DataFrame(crypto_scaled, columns=['price_change_percentage_24h', 'price_change_percentage_7d', 'price_change_percentage_14d', 'price_change_percentage_30d', 'price_change_percentage_60d', 'price_change_percentage_200d',	'price_change_percentage_1y'])
crypto_transformed

# Copy the crypto names from the original data
# This will return a list of all the coin names from the original data
coins = list(df_market_data.index)

# Set the coinid column as index
crypto_transformed['coin_id'] = coins
crypto_transformed = crypto_transformed.set_index('coin_id')

# Display sample data of the prepared dataframe with the index value
crypto_transformed.head(5).style.set_table_styles([df_header_row])


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


### Find the Best Value for k Using the Original Data.

In [9]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 11))
k_values

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [10]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for i in k_values:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(crypto_transformed)
    inertia.append(k_model.inertia_)

In [11]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k_values, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_plot = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow_plot.style.hide().set_table_styles([df_header_row])

k,inertia
1,287.000000
2,195.820218
3,123.190482
4,79.022435
5,65.405923
6,52.933559
7,47.983124
8,37.288187
9,33.061685
10,28.779752


In [12]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
first_elbow_curve = df_elbow_plot.hvplot.line(
    x="k",
    y="inertia",
    title= "Elbow Curve",
    xticks=k_values
)
first_elbow_curve

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** Based on the given clusters, the best value for `k` is *4*.

### Cluster Cryptocurrencies with K-means Using the Original Data

In [13]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [14]:
# Fit the K-Means model using the scaled data
model.fit(crypto_transformed)

KMeans(n_clusters=4, random_state=1)

In [15]:
# Predict the clusters to group the cryptocurrencies using the scaled data
k_labels = model.predict(crypto_transformed)

# Print the resulting array of cluster values.
k_labels

array([3, 3, 1, 1, 3, 3, 3, 3, 3, 1, 1, 1, 1, 3, 1, 3, 1, 1, 3, 1, 1, 3,
       1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 0, 3, 1, 1, 2, 1, 1, 1, 1],
      dtype=int32)

In [16]:
# Create a copy of the DataFrame
clusters_predicted = crypto_transformed.copy()

In [17]:
# Add a new column to the DataFrame with the predicted clusters
clusters_predicted['predicted_cluster'] = k_labels

# Display sample data
clusters_predicted.head().style.set_table_styles([df_header_row])

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,predicted_cluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,3
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,3
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,1
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,1
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,3


In [18]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
cluster_plot = clusters_predicted.hvplot.scatter(
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    by='predicted_cluster',
    hover_cols = 'coin_id',
    title="Crypto Clusters, k=4"
)
cluster_plot

:NdOverlay   [predicted_cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

### Optimize Clusters with Principal Component Analysis.

In [19]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components = 3)
pca

PCA(n_components=3)

In [24]:
# Use the PCA model with `fit_transform` to reduce to three principal components.
pca = PCA(n_components = 3)
pca_data = pca.fit_transform(crypto_transformed)
df_pca = pd.DataFrame(pca_data, columns = ['PC1', 'PC2', 'PC3'])

# Set the coinid column as index
df_pca['coin_id'] = coins
df_pca = df_pca.set_index('coin_id')

# View the first five rows of the DataFrame with the index column
df_pca.head().style.set_table_styles([df_header_row])

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


In [25]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
total_explained_variance = pca.explained_variance_ratio_
total_explained_variance

array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** Based on the sample variances of all individual variables using the PCA model above, the total explained variance is around *89%* with the three principal components. This is obtained by adding the sample variances in the above array. 

In [26]:
# Create a new DataFrame with the PCA data.
pca_df = pd.DataFrame(pca_data, columns=['PC1', 'PC2', 'PC3'])

# Creating a DataFrame with the PCA data

# Copy the crypto names from the original data
pca_df['coin_id'] = df_market_data.index

# Set the coinid column as index
pca_df.set_index('coin_id', inplace = True)

# Display sample data
pca_df.head().style.set_table_styles([df_header_row])


,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


### Find the Best Value for k Using the PCA Data

In [27]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 11))

In [28]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k_values:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(pca_df)
    inertia.append(k_model.inertia_)

In [29]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k_values, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca_plot = pd.DataFrame(elbow_data)

# Display the DataFrame
df_elbow_pca_plot.style.hide().set_table_styles([df_header_row])

k,inertia
1,256.874086
2,165.901994
3,93.774626
4,49.665497
5,38.352251
6,27.618972
7,21.134056
8,17.437664
9,13.742792
10,10.484890


In [30]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_curve_pca = df_elbow_pca_plot.hvplot.line(
    x="k",
    y="inertia",
    title= "Elbow Curve using PCA Data",
    xticks=k_values,
    line_color="r")\
    .opts(bgcolor='#ddd')
elbow_curve_pca

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?
    * **Answer:** The best value for `k` using the PCA data is *4*, as evidenced by the elbow curve above. 

* **Question:** Does it differ from the best k value found using the original data?
    * **Answer:** No, it *does not* differ from the best `k` value found using the original data. 

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [34]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [35]:
# Fit the K-Means model using the PCA data
model.fit(pca_df)

KMeans(n_clusters=4, random_state=1)

In [36]:
# Predict the clusters to group the cryptocurrencies using the PCA data
k = model.predict(pca_df)
# Print the resulting array of cluster values.
print(k)

[1 1 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 2 1 0 0 3
 0 0 0 0]


In [37]:
# Create a copy of the DataFrame with the PCA data
pca_df_copy = pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
pca_df_copy['predicted_cluster'] = k

# Display sample data
pca_df_copy.head().style.set_table_styles([df_header_row])

,PC1,PC2,PC3,predicted_cluster
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,1
ethereum,-0.458261,0.458466,0.952877,1
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,1


In [38]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_cluster = pca_df_copy.hvplot.scatter(
    x="PC1",
    y="PC2",
    by = 'predicted_cluster',
    hover_cols='coin_id',
    title="Crypto Clusters with PCA, k=4")\
      .opts(bgcolor='#ddd')
pca_cluster

:NdOverlay   [predicted_cluster]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [39]:
# Composite plot to contrast the Elbow curves
composite_elbow_curves = first_elbow_curve * elbow_curve_pca
composite_elbow_curves

:Overlay
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [40]:
# Composite plot to contrast the clusters
composite_contrast_clusters = (cluster_plot + pca_cluster).cols(1)
composite_contrast_clusters

:Layout
   .NdOverlay.I  :NdOverlay   [predicted_cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [predicted_cluster]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?
    * **Answer:** Based on the cluster analysis results, fewer features did not have an impact on the number of clusters `k = 4` as evidenced by the elbow curve plots. In addition, based on the scatter plots, the crypto clusters 1 and 2 seem to be very different from the rest of the crypto coins as clusters 0 and 3 seem to be more similar. PCA Model resulted in more tighter clusters than that of the original analysis. 